In [3]:
import json
import urllib.request
import pandas as pd
from pandas.io.json import json_normalize

In [4]:
QUERY_URL = "https://www.alphavantage.co/query?function={REQUEST_TYPE}&apikey={KEY}&symbol={SYMBOL}"
API_KEY = 'RG3MY4ZIWNEBVF8F'

def _request(symbol, req_type):
    with urllib.request.urlopen(QUERY_URL.format(REQUEST_TYPE=req_type, KEY=API_KEY, SYMBOL=symbol)) as req:
        data = req.read().decode("UTF-8")
    return data

def get_monthly_data(symbol):
    return json.loads(_request(symbol, 'TIME_SERIES_MONTHLY'))

def get_daily_data(symbol):
    return json.loads(_request(symbol, 'TIME_SERIES_DAILY'))

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [191]:
company_df = pd.read_excel('company_for_priceindex.xls')
company_df.loc[company_df.company == 'Google','company_index'] = 'GOOGL'
company_df.to_csv('company_for_priceindex.xls', index=False)

#### Create the first one

In [143]:
company_list =  company_df.company_index.tolist()

company_data_daily = pd.DataFrame(columns = ['yyyy-mm-dd'])
for company in company_list:
    df = pd.DataFrame.from_dict(json_normalize(get_daily_data(company)))
    cols_to_use = [i for i in df.columns if 'Daily' in i and 'close' in i] #data at close
    cols_to_use_renamed = {i:i.replace('Time Series (Daily).', '')[:10] for i in cols_to_use}

    df = df.rename(columns = cols_to_use_renamed)[cols_to_use_renamed.values()].T
    data = pd.DataFrame(columns = ['yyyy-mm-dd', company])
    data[company] = df[0].tolist()
    data['yyyy-mm-dd'] = df.index.tolist()

    company_data_daily = company_data_daily.merge(data, how = 'outer', on = 'yyyy-mm-dd')
    
comp_notnull = company_data_daily.loc[:,company_data_daily.isnull().sum() != company_data_daily.shape[0]].columns.tolist()
company_list_remaining = [i for i in company_list if i not in comp_notnull]

In [144]:
company_data_daily = company_data_daily.loc[:,comp_notnull]

#### Repeat a few times

In [187]:
for company in company_list_remaining:
    df = pd.DataFrame.from_dict(json_normalize(get_daily_data(company)))
    cols_to_use = [i for i in df.columns if 'Daily' in i and 'close' in i] #data at close
    cols_to_use_renamed = {i:i.replace('Time Series (Daily).', '')[:10] for i in cols_to_use}

    df = df.rename(columns = cols_to_use_renamed)[cols_to_use_renamed.values()].T
    data = pd.DataFrame(columns = ['yyyy-mm-dd', company])
    data[company] = df[0].tolist()
    data['yyyy-mm-dd'] = df.index.tolist()

    company_data_daily = company_data_daily.merge(data, how = 'outer', on = 'yyyy-mm-dd')
    
comp_notnull = company_data_daily.loc[:,company_data_daily.isnull().sum() != company_data_daily.shape[0]].columns.tolist()
company_list_remaining = [i for i in company_list if i not in comp_notnull]

In [188]:
company_data_daily = company_data_daily.loc[:,comp_notnull]

In [189]:
company_data_daily.shape

(100, 41)

In [190]:
company_data_daily.to_csv('company_financial_data_41.csv', index=False)